In [199]:
import pandas as pd
import datetime
import re

In [200]:
def date_range(start, end):
    start = datetime.datetime.strptime(start, "%Y-%m-%d")
    end = datetime.datetime.strptime(end, "%Y-%m-%d")
    dates = [date.strftime("%Y-%m-%d") for date in pd.date_range(start, periods=(end-start).days+1)]
    return dates

In [201]:
catch = pd.read_csv('고등어 일일 어획량1(kg).csv', encoding='utf-8-sig')
catch.rename(columns={'위판일자':'일시'},inplace=True)
weather = pd.read_csv('남해 기후데이터.csv',encoding='utf-8-sig')
oil = pd.read_csv("유가일일데이터.csv",encoding='utf-8-sig')
dates = date_range('2013-01-01','2023-04-04')
date = pd.DataFrame(dates, columns=['일시'])


In [202]:
rawData = date.merge(catch, on='일시', how='outer')
rawData = rawData.merge(weather,on='일시',how='outer')
rawData = rawData.merge(oil,on='일시',how='outer')
rawData['일시'] = pd.to_datetime(rawData['일시'])
rawData = rawData.set_index('일시')


In [204]:
rawData.isnull().sum()


위판중량             64
품종               64
평균 풍속(m/s)        2
평균기압(hPa)         2
평균 상대습도(%)        2
평균 기온(°C)         2
평균 수온(°C)         2
평균 최대 파고(m)       2
평균 유의 파고(m)       2
평균 파주기(sec)       2
WTI            1165
dtype: int64

In [205]:
rawData = rawData.interpolate(method='time',limit_area='inside')
rawData.isnull().sum()

위판중량           35
품종             64
평균 풍속(m/s)      0
평균기압(hPa)       0
평균 상대습도(%)      0
평균 기온(°C)       0
평균 수온(°C)       0
평균 최대 파고(m)     0
평균 유의 파고(m)     0
평균 파주기(sec)     0
WTI             1
dtype: int64

In [206]:
rawData['품종'] = rawData['품종'].fillna('고등어')
rawData['WTI'] = rawData['WTI'].fillna('91.82') #12년 12월 31일 종가
rawData.isnull().sum()

위판중량           35
품종              0
평균 풍속(m/s)      0
평균기압(hPa)       0
평균 상대습도(%)      0
평균 기온(°C)       0
평균 수온(°C)       0
평균 최대 파고(m)     0
평균 유의 파고(m)     0
평균 파주기(sec)     0
WTI             0
dtype: int64

In [207]:
rawData.isnull().sum()
rawData = rawData.rename(columns={'위판중량':'위판중량(kg)'})

rawData

위판중량(kg)   품종  평균 풍속(m/s)    평균기압(hPa)  평균 상대습도(%)  평균 기온(°C)  \
일시                                                                          
2013-01-01    5766.0  고등어    7.766667  1020.866667   56.333333   7.266667   
2013-01-02  253212.0  고등어    9.266667  1027.300000   57.666667   5.166667   
2013-01-03  119183.0  고등어    9.800000  1033.866667   59.666667   0.033333   
2013-01-04  128879.0  고등어    7.133333  1029.200000   54.333333   1.366667   
2013-01-05    3912.0  고등어    7.366667  1024.600000   53.666667   3.300000   
...              ...  ...         ...          ...         ...        ...   
2023-03-31       NaN  고등어    5.057143  1018.000000   74.142857  15.028571   
2023-04-01       NaN  고등어    6.037500  1015.687500   83.125000  15.662500   
2023-04-02       NaN  고등어    8.800000  1017.575000   74.500000  15.262500   
2023-04-03       NaN  고등어   10.587500  1021.025000   77.750000  14.875000   
2023-04-04       NaN  고등어    8.925000  1018.275000   85.000000  16.300000   

            평균 수온(°C)  평균 최대 파고(m)  평균 유의 파고(m)  평균 파주기(sec)        WTI  
일시                                                                       
2013-01-01  14.533333     2.266667     1.500000     6.266667      91.82  
2013-01-02  14.833333     2.366667     1.533333     5.400000      93.12  
2013-01-03  14.400000     2.400000     1.533333     5.366667      92.92  
2013-01-04  14.133333     1.866667     1.166667     6.033333      93.09  
2013-01-05  14.100000     1.466667     0.933333     5.500000  93.123333  
...               ...          ...          ...          ...        ...  
2023-03-31  14.771429     1.557143     0.928571     5.642857      75.67  
2023-04-01  15.287500     1.812500     1.037500     5.450000  77.253333  
2023-04-02  15.050000     3.037500     1.850000     6.025000  78.836667  
2023-04-03  15.175000     3.737500     2.375000     6.612500      80.42  
2023-04-04  15.225000     2.887500     1.787500     5.912500      80.71  

[3748 rows x 11 columns]

In [208]:
rawData= rawData[['품종','위판중량(kg)','평균 풍속(m/s)','평균기압(hPa)','평균 상대습도(%)','평균 기온(°C)','평균 수온(°C)','평균 최대 파고(m)','평균 유의 파고(m)','평균 파주기(sec)',	'WTI']]

In [209]:
rawData.to_csv('임시.csv',encoding='utf-8-sig')